In [2]:
import pandas as pd
import joblib

df_steam_web = joblib.load(r"..\assets\SteamWeb\steamWeb_processed.pkl")
df_RAWG = joblib.load(r"..\assets\RAWG\RAWG_clean.pkl")
df_steam_spy = joblib.load(r"..\assets\SteamSpy\SteamSpy_clean.pkl")

print(df_steam_web.shape, df_RAWG.shape, df_steam_spy.shape)

(3985828, 8) (62138, 31) (69226, 18)


In [3]:
df_steam_web.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3985828 entries, 0 to 3985827
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             3985828 non-null  object 
 1   app_id              3985828 non-null  int64  
 2   relevance_score     3985828 non-null  float64
 3   locCountryCode      3985828 non-null  object 
 4   has_coordinates     3985828 non-null  bool   
 5   latitude            3075256 non-null  float64
 6   longitude           3075256 non-null  float64
 7   account_age_months  3985828 non-null  float64
dtypes: bool(1), float64(4), int64(1), object(2)
memory usage: 216.7+ MB


In [4]:
df_RAWG.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62138 entries, 0 to 704056
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           62138 non-null  Int64  
 1   slug                         62138 non-null  string 
 2   name                         62138 non-null  string 
 3   tba                          62138 non-null  boolean
 4   weighted_avg_RAWG_rating     62138 non-null  Float64
 5   ratings_count                62138 non-null  Int64  
 6   reviews_text_count           62138 non-null  Int64  
 7   RAWG_bookmark_count          62138 non-null  Int64  
 8   metacritic_rating            4247 non-null   Int64  
 9   steam_avg_playtime(h)        62138 non-null  Int64  
 10  RAWG_system_suggest_count    62138 non-null  Int64  
 11  reviews_count                62138 non-null  Int64  
 12  genres                       62138 non-null  object 
 13  tags                

In [5]:
df_RAWG.drop(columns=['id', 'slug'], inplace=True)

In [6]:
df_steam_spy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69226 entries, 570 to 449420
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   appid                  69226 non-null  Int64  
 1   name                   69226 non-null  string 
 2   developer              69226 non-null  object 
 3   publisher              69226 non-null  object 
 4   score_rank             69226 non-null  int64  
 5   positive               69226 non-null  Int64  
 6   negative               69226 non-null  Int64  
 7   userscore              69226 non-null  Int64  
 8   average_forever        69226 non-null  Int64  
 9   average_2weeks         69226 non-null  Int64  
 10  median_forever         69226 non-null  Int64  
 11  median_2weeks          69226 non-null  Int64  
 12  price                  69215 non-null  float64
 13  initialprice           69217 non-null  float64
 14  discount               69217 non-null  float64
 15  conc

In [7]:
df_RAWG[df_RAWG.duplicated(subset=['steam_app_id'], keep=False)][['name', 'steam_app_id']].sort_values('steam_app_id')

,name,steam_app_id
166,Counter-Strike: Condition Zero,80
314,Counter-Strike: Condition Zero Deleted Scenes,80
6250,Counter-Strike 2,730
3,Counter-Strike: Global Offensive,730
28660,Quake II: Enhanced Edition,2320
...,...,...
121899,INVESTIGATOR TRIALS,2756320
41106,FANTASIAN Neo Dimension,2844850
20942,Fantasian,2844850
31335,Teenage Mutant Ninja Turtles: Splintered Fate,2996040


In [8]:
df_RAWG = df_RAWG.drop_duplicates(subset=['steam_app_id'])
df_RAWG['steam_app_id'] = df_RAWG['steam_app_id'].astype(int)
df = pd.merge(df_steam_web, df_RAWG, how='inner', left_on="app_id", right_on='steam_app_id')
df.shape

(3598422, 37)

In [9]:
df.duplicated(subset=['user_id', 'app_id']).sum()

np.int64(0)

In [10]:
df = pd.merge(df, df_steam_spy, how='inner', left_on="app_id", right_on="appid")
df.shape

(3278536, 55)

In [11]:
df.duplicated(subset=['user_id', 'app_id']).sum()

np.int64(0)

In [12]:
df.drop(columns=[
    'steam_app_id', 'appid', 'name_y',  # Duplicate columns
    'score_rank', # Redundant with user_score
    'steam_avg_playtime(h)' # Redundant with average_forever
    ], inplace=True)

df.rename(columns={
    'name_x':'game_name',
    'locCountryCode': 'user_country_code',
    'has_coordinates': 'user_has_coordinates',
    'latitude': 'user_latitude',
    'longitude': 'user_longitude',
    'account_age_months': 'user_account_age_months',
    'tba': 'game_tba',
    'weighted_avg_RAWG_rating': 'game_RAWG_weighted_avg_rating',
    'ratings_count': 'game_RAWG_ratings_count',
    'reviews_text_count': 'game_RAWG_reviews_with_text_count',
    'RAWG_bookmark_count': 'game_RAWG_bookmark_count',
    'metacritic_rating': 'game_metacritic_rating',
    'steam_avg_playtime(h)': 'game_steam_avg_playtime(h)',
    'RAWG_system_suggest_count': "game_RAWG_system_suggest_count",
    'reviews_count': 'game_RAWG_reviews_count',
    'genres': 'game_genres',
    'tags': 'game_tags',
    'esrb_rating': 'game_esrb_rating',
    'released_year': 'game_released_year',
    'released_month': 'game_released_month',
    'released_day': 'game_released_day',
    'RAWG_rating_5_percent': 'game_RAWG_rating_5_percent',
    'RAWG_rating_4_percent': 'game_RAWG_rating_4_percent',
    'RAWG_rating_3_percent': 'game_RAWG_rating_3_percent',
    'RAWG_rating_1_percent': 'game_RAWG_rating_1_percent',
    'bookmark_type_yet_count': 'game_RAWG_bookmark_type_yet_count',
    'bookmark_type_owned_count': 'game_RAWG_bookmark_type_owned_count',
    'bookmark_type_beaten_count': 'game_RAWG_bookmark_type_beaten_count',
    'bookmark_type_toplay_count': 'game_RAWG_bookmark_type_toplay_count',
    'bookmark_type_dropped_count': 'game_RAWG_bookmark_type_dropped_count',
    'bookmark_type_playing_count': 'game_RAWG_bookmark_type_playing_count',
    'available_platform': 'game_available_platform',
    'available_parent_platforms': 'game_available_parent_platforms',
    'developer': 'game_developer',
    'publisher': 'game_publisher',
    'positive': 'game_positive_review_count',
    'negative': 'game_negative_review_count',
    'userscore': 'game_avg_user_score',
    'average_forever': 'game_avg_playtime_forever',
    'average_2weeks': 'game_avg_playtime_last_2weeks',
    'median_forever': 'game_median_playtime_forever',
    'median_2weeks': 'game_median_last_2weeks',
    'price': 'game_current_price',
    'initialprice': 'game_initial_price',
    'discount': 'game_current_discount',
    'concurrent_user': 'game_concurrent_user',
    'estimate_owners_lower': 'game_estimate_owners_lower',
    'estimate_owners_upper': 'game_estimate_owners_upper'
    }, inplace=True)

In [13]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3278536 entries, 0 to 3278535
Data columns (total 50 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   user_id                                3278536 non-null  object 
 1   app_id                                 3278536 non-null  int64  
 2   relevance_score                        3278536 non-null  float64
 3   user_country_code                      3278536 non-null  object 
 4   user_has_coordinates                   3278536 non-null  bool   
 5   user_latitude                          2537747 non-null  float64
 6   user_longitude                         2537747 non-null  float64
 7   user_account_age_months                3278536 non-null  float64
 8   game_name                              3278536 non-null  string 
 9   game_tba                               3278536 non-null  boolean
 10  game_RAWG_weighted_avg_rating          327

In [14]:
print(len(df['user_id'].unique()), len(df['app_id'].unique()))

4041 24751


In [15]:
df.duplicated(subset=['user_id', 'app_id']).sum()

np.int64(0)

In [16]:
# Train-test split by user_id
from sklearn.model_selection import train_test_split

unique_users = df["user_id"].unique()
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)

train_df = df[df["user_id"].isin(train_users)]
test_df = df[df["user_id"].isin(test_users)]

joblib.dump(train_df, r"..\assets\combined\train_raw.pkl")
joblib.dump(test_df, r"..\assets\combined\test_raw.pkl")

print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")

Train size: 2595694, Test size: 682842
